In [5]:
import import_ipynb
import os

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

import scipy.stats as stats
from scipy.stats import chi2_contingency
import seaborn as sns

### Import data

In [6]:
os.getcwd()

'/Users/berlincheng/Documents/GitHub/native_search_redesign'

In [7]:
import import_data

In [8]:
filepath='/Users/berlincheng/Documents/GitHub/native_search_redesign/search_redesign_dataset_2.sql'
data = import_data.fpath_to_df(filepath)

--USE TASKRABBIT_PRODUCTION.PUBLIC;
SELECT 		O.GUID AS "guid"
			,O.VARIATION AS "variation"
			,IFNULL(SEARCH_BFS,0) AS "search_bfs" 
			,IFNULL(BFS,0) AS "bfs"
			,IFNULL(BOOKED,0) AS "booked"
--;SELECT COUNT(DISTINCT O.GUID)
FROM 
(
	--Get guid-level info from Optimizely table
	SELECT 		GUID
				,CASE WHEN VARIATION_ID LIKE '%20041998218%' THEN 'control' WHEN VARIATION_ID LIKE '%20059568078%' THEN 'search_update' END AS VARIATION
	FROM 		PUBLIC.OPTIMIZELY_ENR_DECI 
	WHERE 		TRUE 
				--AND TIMESTAMP BETWEEN '2021-03-22' AND '2021-04-12'
				AND	EXPERIMENT_ID LIKE '%20063077810%'
				AND ENVIRONMENT ='production'
	GROUP BY 	1,2
) O 
LEFT JOIN 	
(
	--Get guid-level info from react_native table
	SELECT 		GUID AS GUID
				,MAX(IFF(CLIENT_PUBLISH_TYPE='TaskSearch-Presented',1,0)) AS SAW_SEARCH --Was shown the search results page
				,MAX(IFF(CLIENT_PUBLISH_TYPE='TaskSearch-Result-Selected',1,0)) AS SEARCH_BFS
				,MAX(IFF(CLIENT_PUBLISH_TYPE IN('Funnel-Created','QuickFormAddressEditor-Pr

In [9]:
data.shape

(64181, 5)

In [10]:
data.dtypes

guid          object
variation     object
search_bfs     int64
bfs            int64
booked         int64
dtype: object

In [11]:
data.head()

,guid,variation,search_bfs,bfs,booked
0,39c97861-b376-46ce-a151-61ab0e70fa2a,search_update,1,1,0
1,1ad13b3b-a101-4fa3-8df5-45787140f66c,search_update,1,1,1
2,89a5b955-1da6-7151-13f1-d18a09cd7b4b,search_update,1,1,1
3,88829d66-a44d-713b-d3e0-31700f96afdb,search_update,1,1,0
4,88829d66-a44d-713b-d3e0-31700f96afdb,control,1,1,0


### Data Manipulation

In [12]:
# Check for null values
print(len(data[data.search_bfs.isna()]))
print(len(data[data.bfs.isna()]))
print(len(data[data.booked.isna()]))

0
0
0


### EDA

In [13]:
# Split into control vs. search update variants
control=data[data.variation=='control']
search_update=data[data.variation=='search_update']

print(len(control),'+',len(search_update),'=',len(control)+len(search_update))
print(len(data))

32111 + 32070 = 64181
64181


### Run Mann-Whitney Test

In [14]:
# mann_whitney function that inputs the control + variant conversion columns
def mann_whitney(control,variant,conv):
    u_statistic, p_value = stats.mannwhitneyu(control[conv],variant[conv])
    print("u_statistic: \t",u_statistic,"\np_value: \t",p_value)
    if p_value<0.05:
        print("\n Mann-Whitney test is significant for ",conv)

In [15]:
mann_whitney(control,search_update,'search_bfs')

u_statistic: 	 509901410.0 
p_value: 	 0.0003735694792956129

 Mann-Whitney test is significant for  search_bfs


In [16]:
mann_whitney(control,search_update,'bfs')

u_statistic: 	 510837482.5 
p_value: 	 4.2913103124285835e-06

 Mann-Whitney test is significant for  bfs


In [17]:
# mann_whitney(control,search_update,'booked')

### Run chi-square test

In [18]:
def chi2_test(variations,conv):    
    contingency = pd.crosstab(variations, data[conv]) 
    display(contingency)
    
    contingency_pct = pd.crosstab(variations, data[conv], normalize='index')
    display(contingency_pct)
    
#     plt.figure(figsize=(4,2)) 
#     sns.heatmap(contingency, annot=True, cmap="YlGnBu")
    
    c, p, dof, expected = chi2_contingency(contingency)
    print('\np-value:\t',p)
    if p<0.05:
        print("\n Chi-square test is significant for",conv)

In [19]:
chi2_test(data['variation'],'search_bfs')

search_bfs,0,1
variation,,
control,5228,26883
search_update,4910,27160


search_bfs,0,1
variation,,
control,0.162810,0.837190
search_update,0.153103,0.846897



p-value:	 0.0007769642383121018

 Chi-square test is significant for search_bfs


In [20]:
chi2_test(data['variation'],'bfs')

bfs,0,1
variation,,
control,1584,30527
search_update,1835,30235


bfs,0,1
variation,,
control,0.049329,0.950671
search_update,0.057219,0.942781



p-value:	 9.311698358232917e-06

 Chi-square test is significant for bfs


In [21]:
# chi2_test(data['variation'],'booked')

### Run Fishers Test

In [22]:
def fisher_exact(variations,conv):    
    contingency = pd.crosstab(variations, data[conv]) 
    display(contingency)
    
    oddsratio, p_value = stats.fisher_exact(contingency)

    print('\np-value:\t',p_value)
    if p_value<0.05:
        print("\n Fisher exact test is significant for",conv)

In [23]:
fisher_exact(data['variation'],'search_bfs')

search_bfs,0,1
variation,,
control,5228,26883
search_update,4910,27160



p-value:	 0.0007621460540294376

 Fisher exact test is significant for search_bfs


In [24]:
fisher_exact(data['variation'],'bfs')

bfs,0,1
variation,,
control,1584,30527
search_update,1835,30235



p-value:	 8.65625179513771e-06

 Fisher exact test is significant for bfs


In [25]:
fisher_exact(data['variation'],'booked')

booked,0,1
variation,,
control,23049,9062
search_update,22952,9118



p-value:	 0.5572434120520017


### Calculate lift across search BFS, overall BFS, and booking rates

In [26]:
round(3.23432,2)

3.23

In [64]:
# calculate_lift function that inputs the control + variant dfs
def calculate_lift(control,variant,conv):
    control_conv=100*(control[conv].sum()/control['guid'].count())
    print('CONTROL')
    print('total visitors:',control['guid'].count())
    print('unique conversions:',control[conv].sum())
    print('control ',conv,' rate:','%.2f'%control_conv)

    print('\nVARIANT')
    print('total visitors:',variant['guid'].count())
    print('unique conversions:',variant[conv].sum())
    variant_conv=100*(variant[conv].sum()/variant['guid'].count())
    print('variant ',conv,' rate:','%.2f'%variant_conv)
    
    print('\nLIFT - CONTROL VS. VARIANT')
    lift_magnitude = variant_conv-control_conv
    lift_pct = 100*(lift_magnitude/control_conv)
#     print('Lift (magnitude) in',conv,':','%.2f'%lift_magnitude)
    print('Lift (percent) in',conv,':','%.2f'%lift_pct,'percent','(','%.2f'%control_conv,'vs.','%.2f'%variant_conv,')')

In [65]:
calculate_lift(control,search_update,'search_bfs')

CONTROL
total visitors: 32111
unique conversions: 26883
control  search_bfs  rate: 83.72

VARIANT
total visitors: 32070
unique conversions: 27160
variant  search_bfs  rate: 84.69

LIFT - CONTROL VS. VARIANT
Lift (percent) in search_bfs : 1.16 percent ( 83.72 vs. 84.69 )


In [66]:
calculate_lift(control,search_update,'bfs')

CONTROL
total visitors: 32111
unique conversions: 30527
control  bfs  rate: 95.07

VARIANT
total visitors: 32070
unique conversions: 30235
variant  bfs  rate: 94.28

LIFT - CONTROL VS. VARIANT
Lift (percent) in bfs : -0.83 percent ( 95.07 vs. 94.28 )


In [67]:
calculate_lift(control,search_update,'booked')

CONTROL
total visitors: 32111
unique conversions: 9062
control  booked  rate: 28.22

VARIANT
total visitors: 32070
unique conversions: 9118
variant  booked  rate: 28.43

LIFT - CONTROL VS. VARIANT
Lift (percent) in booked : 0.75 percent ( 28.22 vs. 28.43 )
